In [1]:
## ----------- note -----------
## 9월 15-16일 기준 라즈베리 파이에서 영상, CS 받아오는거 딜레이 없이 꽤 잘됨.
## 모터값 쏘는 부분은 오른쪽 바퀴 사망이슈로 인해 패킷 살짝 변경됨.
## 그냥 모터값만 주는게 아니라 방향 제어값(MF, MR, ML, MS)이랑 왼쪽 모터값, 오른쪽 모터값 따로 주는걸로. 모터값은 이왕이면 오른쪽이 더 높게 40, 47 추천.
## 영상 수신받는 부분 살짝 변경됨 원래는 pickle 사용해서 프레임 직렬화 시켜서 송신했지만 피클은 딜레이가 심해져서 jpeg형태를 image encode해서 송신하는 방법으로 변경됐기에 서버도 image decode하는 방법으로 변경.
## -----------------------------

import numpy as np
import socket
import struct
import cv2
import threading
import time
import queue

# Global queue for frames
frame_queue = queue.Queue(maxsize=10)  # Limit queue size

def handle_status(data):
    try:
        if len(data) == 5:  # Header (2 bytes) + status (2 bytes)
            command_id, status_value = struct.unpack("2sh", data[:4]) #end byte 제거를 위한 slicing
            if command_id == b'CS':
                if status_value == 1:
                    print("All connections are available.") #연결상태 정상이면 GUI에 정상이라고 보내주시면 될듯합니다.
                elif status_value == 0:
                    print("One or more connections are missing.") #이건 비정상 상태로?
                else:
                    print("Unknown status received.")
    except Exception as e:
        print(f"Error handling status data: {e}")

def handle_frame(conn):
    try:
        # Receive frame size
        size_data = b''
        while len(size_data) < 4:
            packet = conn.recv(4 - len(size_data))
            if not packet:
                return
            size_data += packet
        
        frame_size = struct.unpack('>L', size_data)[0]

        # Receive frame data
        frame_data = b''
        while len(frame_data) < frame_size:
            packet = conn.recv(frame_size - len(frame_data))
            if not packet:
                return
            frame_data += packet
        
        conn.recv(1) # receive and discard the '\n' end byte

        frame = cv2.imdecode(np.frombuffer(frame_data, np.uint8), cv2.IMREAD_COLOR)
        
        # Use a queue with a size limit to manage frame buffer
        if frame_queue.full():
            frame_queue.get()  # Discard oldest frame if queue is full
        frame_queue.put(frame)

        # Display the frame
        while not frame_queue.empty():
            frame_to_display = frame_queue.get()
            cv2.imshow("Received Frame", frame_to_display)
            cv2.waitKey(1)

    except Exception as e:
        print(f"Error handling frame: {e}")


def send_motor_cmd(sock):
    while True:
        #command_ids = ['MF', 'MR', 'ML', 'MS'] #move front, right, left, stop
        #command_id = random.choice(command_ids) #모터 제어 테스트시 사용바람.
        command_id = 'MF' #예시
        left_speed = 40   #예시. 255가 최대이나 100을 넘기지 말것.
        right_speed = 47  #예시. 우리 테스트 환경에선 40-50대도 빠른편.

        ## 위에는 상곤없는데 모터 보내는거는 꼭!!! 2shh 형태로 보내주세요... 2sii나 L이나 다른건 다 안먹히더라구요..
        cmd_packet = struct.pack('2shh', command_id.encode('utf-8'), left_speed, right_speed)
        cmd_packet += b'\n'
        print(f"Sending motor command: {command_id} with speeds L: {left_speed}, R: {right_speed}")

        try:
            sock.sendall(cmd_packet)
        except Exception as e:
            print(f"Error sending motor command: {e}")
        
        time.sleep(4)  # 2~5초에 한번 쏴주는것만으로도 충분함. 타임 슬립없으면 충돌남.

def handle_client(conn):
    try:
        while True:
            # Receive header
            header = b''
            while len(header) < 2:
                packet = conn.recv(2 - len(header))
                if not packet:
                    return
                header += packet

            if header == b'CS':  # Connection status
                status_data = b''
                while len(status_data) < 3:
                    packet = conn.recv(3 - len(status_data))
                    if not packet:
                        return
                    status_data += packet
                handle_status(header + status_data[:-1])

            elif header == b'SF':  # Video frame
                handle_frame(conn)
            else:
                print(f"Unknown header received: {header}")
                # Attempt to resynchronize
                conn.recv(1024)  # 싱크 맞추기 위해 쓰레기값 버리기

    except Exception as e:
        print(f"Error handling client: {e}")

def main(server_ip, server_port):
    server_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_sock.bind((server_ip, server_port))
    server_sock.listen(1)

    print(f"Server listening on {server_ip}:{server_port}. Waiting for client")

    conn, addr = server_sock.accept()
    print(f"Connection from {addr}")

    client_thread = threading.Thread(target=handle_client, args=(conn,))
    client_thread.start()

    motor_cmd_thread = threading.Thread(target=send_motor_cmd, args=(conn,))
    motor_cmd_thread.start()

    client_thread.join()
    motor_cmd_thread.join()

    conn.close()
    server_sock.close()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main('192.168.0.13', 3141)

Server listening on 192.168.0.13:3141. Waiting for client
Connection from ('192.168.0.44', 55202)
Sending motor command: MF with speeds L: 40, R: 47


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/jh/venv/opencv/lib/python3.10/site-packages/cv2/qt/plugins"


Sending motor command: MF with speeds L: 40, R: 47
Sending motor command: MF with speeds L: 40, R: 47
Sending motor command: MF with speeds L: 40, R: 47
Sending motor command: MF with speeds L: 40, R: 47
Sending motor command: MF with speeds L: 40, R: 47
Sending motor command: MF with speeds L: 40, R: 47
Sending motor command: MF with speeds L: 40, R: 47
Error handling client: [Errno 104] Connection reset by peer
Sending motor command: MF with speeds L: 40, R: 47
Error sending motor command: [Errno 32] Broken pipe
Sending motor command: MF with speeds L: 40, R: 47
Error sending motor command: [Errno 32] Broken pipe
Sending motor command: MF with speeds L: 40, R: 47
Error sending motor command: [Errno 32] Broken pipe
Sending motor command: MF with speeds L: 40, R: 47
Error sending motor command: [Errno 32] Broken pipe


: 